In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import configparser
import os

from utils import util_data, util_preprocess


In [4]:
config = configparser.ConfigParser()
config.read("config.ini")


['config.ini']

In [5]:
ds_conf = config["GSMARENA_DS"]
# Download the training and test datasets
dh = util_data.DownloadHelper(
    ds_conf["url"], ds_conf["name"], ds_conf["mode"], quiet=False
)

dh.download()

File gsm.csv already exists. Skip download.


In [6]:
df = dh.read_csv(low_memory=False, sep=",")

In [31]:
from importlib import reload
reload(util_preprocess)

<module 'utils.util_preprocess' from 'c:\\Users\\andre\\Documents\\ai-master\\counterfactual-explanations\\utils\\util_preprocess.py'>

In [27]:
preprocess = util_preprocess.GSMArenaPreprocess(df)
df = preprocess.preprocess()

In [26]:
df = (
    df.pipe(util_preprocess.filter_unwanted_columns)
    .pipe(util_preprocess.preprocess_feature, "misc_price")
    .pipe(util_preprocess.preprocess_feature, "launch_announced")
    .pipe(util_preprocess.preprocess_feature, "display_size")
    .pipe(util_preprocess.preprocess_feature, "display_type")
    .pipe(util_preprocess.preprocess_feature, "display_resolution")
    .pipe(util_preprocess.preprocess_feature, "memory_card_slot")
    .pipe(util_preprocess.preprocess_feature, "sound_loudspeaker")
    .pipe(util_preprocess.preprocess_feature, "sound_3.5mm_jack")
    .pipe(util_preprocess.preprocess_feature, "comms_wlan")
    .pipe(util_preprocess.preprocess_feature, "comms_nfc")
    .pipe(util_preprocess.preprocess_feature, "memory_internal")
    .pipe(util_preprocess.preprocess_feature, "network_technology")
    .pipe(util_preprocess.preprocess_feature, "battery")
    .pipe(util_preprocess.preprocess_feature, "battery_charging")
    .pipe(util_preprocess.preprocess_feature, "body")
    .pipe(util_preprocess.preprocess_feature, "main_camera_cols")
    .pipe(util_preprocess.preprocess_feature, "selfie_camera_cols")    
)


In [28]:
print(f"The dataframe has {df.shape[0]} rows and {df.shape[1]} columns.")

The dataframe has 2647 rows and 21 columns.


In [33]:
for i in df.columns:
    print(f"{i}: {df.loc[:, i].value_counts().index.to_list()}")

misc_price: [200.0, 250.0, 150.0, 130.0, 300.0, 180.0, 120.0, 170.0, 100.0, 160.0, 220.0, 230.0, 190.0, 280.0, 110.0, 140.0, 260.0, 270.0, 350.0, 400.0, 210.0, 240.0, 80.0, 90.0, 330.0, 290.0, 320.0, 70.0, 500.0, 360.0, 450.0, 420.0, 310.0, 380.0, 390.0, 370.0, 340.0, 440.0, 550.0, 60.0, 430.0, 410.0, 480.0, 470.0, 460.0, 700.0, 600.0, 850.0, 650.0, 50.0, 880.0, 530.0, 900.0, 199.99, 620.0, 680.0, 520.0, 1000.0, 670.0, 800.0, 490.0, 1100.0, 640.0, 580.0, 299.99, 690.0, 134.99, 149.99, 1200.0, 204.75, 40.0, 599.0, 730.0, 199.0, 750.0, 145.0, 349.0, 179.99, 570.0, 510.0, 950.0, 329.0, 770.0, 1150.0, 540.0, 179.0, 279.0, 1170.45, 1165.0, 246.99, 505.99, 145.99, 289.99, 233.9, 1087.99, 807.8, 799.99, 219.0, 149.0, 229.5, 820.0, 798.14, 55.0, 92.0, 489.0, 209.9, 699.95, 308.86, 419.0, 579.99, 108.98, 474.4, 149.98, 173.86, 459.99, 112.0, 579.0, 799.0, 830.0, 119.9, 77.83, 1250.0, 960.0, 660.0, 1600.0, 1300.0, 920.0, 444.0, 1050.0, 589.0, 1750.0, 235.0, 760.0, 590.0, 125.0, 9000.0, 5440.0, 8

Filter conditions for recent mobile phones

- launch_announced >= 2014
- display_size in (4, 7.5)
- display_height and display_width > 512

In [521]:
filter_condition = {
    "launch_announced": lambda x: x >= 2014,
    "display_size": lambda x: x >= 4,
    "display_size": lambda x: x <= 7.5,
    "display_height": lambda x: x >= 512,
    "display_width": lambda x: x >= 512,
}
df = util_preprocess.filter_unwanted_rows(df, filter_condition)

Removing unwanted rows and duplicates...
Shape before removing unwanted rows and duplicates: (2647, 21)
Shape after removing unwanted rows and duplicates: (1973, 21)



In [532]:
out_path = os.path.join("data", "processed", ds_conf["name"])
df.to_csv(out_path, index=False)